In [ ]:
# part 1 - captures the photo of user
import cv2
import numpy as np
import time

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,1)
        
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (300, 300))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './azureaifaces/faces/IMG_TEST_' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 9: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

In [ ]:
# 1.

#!pip install azure-cognitiveservices-vision-customvision

In [ ]:
# 2.

project_id = '36ec88e3-0b08-4c99-ab6f-0be8803f2ec9'
cv_key = 'f00a596c2ab748a3a0fc81364492ac0b'
cv_endpoint = 'https://facerectask6-prediction.cognitiveservices.azure.com/'

model_name = 'model1' # this must match the model name you set when publishing your model iteration (it's case-sensitive)!
print('Ready to predict using model {} in project {}'.format(model_name, project_id))

In [ ]:
# 3.

from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Get the test images from the data/vision/test folder
test_folder = os.path.join('azureaifaces','faces')
test_images = os.listdir(test_folder)

# Create an instance of the prediction service
credentials = ApiKeyCredentials(in_headers={"Prediction-key": cv_key})
custom_vision_client = CustomVisionPredictionClient(endpoint=cv_endpoint, credentials=credentials)

# Create a figure to display the results
fig = plt.figure(figsize=(16, 8))

# Get the images and show the predicted classes for each one
print('Classifying images in {} ...'.format(test_folder))
arr = []
for i in range(len(test_images)):
    # Open the image, and use the custom vision model to classify it
    image_contents = open(os.path.join(test_folder, test_images[i]), "rb")
    classification = custom_vision_client.classify_image(project_id, model_name, image_contents.read())
    # The results include a prediction for each tag, in descending order of probability - get the first one
    prediction = classification.predictions[0].tag_name
    arr.append(prediction)
    # Display the image with its predicted class
    img = Image.open(os.path.join(test_folder, test_images[i]))
    a=fig.add_subplot(len(test_images)/3, 3,i+1)
    a.axis('off')
    imgplot = plt.imshow(img)
    a.set_title(prediction)


In [ ]:
#############################################
#Condition to send mail and whatsapp message
#############################################
   
if max(arr,key=arr.count)=="haha":
    import os
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.image import MIMEImage
    from email.mime.multipart import MIMEMultipart
    import smtplib
    import os
    from dotenv import load_dotenv

    load_dotenv()

    MY_EMAIL=os.getenv('MY_EMAIL')
    MY_PASSWORD = os.getenv('MY_PASSWORD')


    def SendMail(ImgFileName):
        with open(ImgFileName, 'rb') as f:
            img_data = f.read()

        msg = MIMEMultipart()
        msg['Subject'] = 'Madhhav Spotted'
        msg['From'] = 'sender@gmail.com'
        msg['To'] = 'receiver@gmail.com'

        text = MIMEText("Madhhav Was Spotted")
        msg.attach(text)
        image = MIMEImage(img_data, name=os.path.basename(ImgFileName))
        msg.attach(image)

        s = smtplib.SMTP('smtp.gmail.com', 587)
        s.ehlo()
        s.starttls()
        s.ehlo()
        s.login(MY_EMAIL, MY_PASSWORD)
        s.sendmail('sender@gmail.com', 'receiver@gmail.com', msg.as_string())
        s.quit()
        print("Mail Sent")
    SendMail('azureaifaces/faces/IMG_TEST_1.jpg')
#############################################
#Condition to create instance and attach ebs
#############################################
    
elif max(arr,key=arr.count)=="madhhav":
    import boto3
    import time
    ec2 = boto3.resource('ec2')
    ec2_client = boto3.client('ec2')
    # create a new EC2 instance
    instances = ec2.create_instances(
        ImageId='ami-00b6a8a2bd28daf19',
        MinCount=1,
        Placement={
           'AvailabilityZone':'ap-south-1a'
        },
        MaxCount=1,
        InstanceType='t2.micro'
    )
    print("Instance Created")
    id1 = str(instances[0])
    id1 = id1[17:-2]
    # create an EBS volume, 20G size
    print("Creating EBS")
    ebs_vol = ec2.create_volume(
        Size=5,
        AvailabilityZone='ap-south-1a'
    )
    print("EBS Created!")
    time.sleep(30)
    id2 = str(ebs_vol)
    id2 = id2[15:-2]

    response= ec2_client.attach_volume(
            Device="/dev/xvdb",
            InstanceId=id1,
            VolumeId=id2
        )
    print("Volume Attached to EC2 Instance!")

    if response['ResponseMetadata']['HTTPStatusCode']== 200:
        ec2_client.get_waiter('volume_in_use').wait(
            VolumeIds=[id2],
            DryRun=False
        )
    print("FINALLY DONEE!")
    
elif max(arr,key=arr.count)=="avanish":
    import pywhatkit as kit    
    print("******************************************Sending Whatsapp Message***********************************")
    kit.sendwhatmsg_instantly(phone_no="+911234567899", message="Hey Face Recognised !!\n")
else:
    print("Haha")